In [13]:
import pyfolio as pf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sys
import pandas_datareader.data as web
from pathlib import Path
import seaborn as sns
import matplotlib
import matplotlib.gridspec as gridspec
import matplotlib.dates as mdates
from matplotlib.ticker import FormatStrFormatter
import matplotlib.colors as colors
import matplotlib.transforms as bbox
from matplotlib import rcParams

import socket
if socket.gethostname() == 'edmunds-mbp.lan':
    %config InlineBackend.figure_format = 'retina'
    
opt_path = '/Users/ebellord/PycharmProjects/opt_trade'
if  opt_path not in sys.path:
    sys.path.append(opt_path)
from option_simulation import OptionSimulation, OptionTrades
from spx_data_update import UpdateSP500Data, SP500Index
from option_utilities import perf_stats
from overlay_presentation import get_fund_assets, daily_hfrx, monthly_hfri, monthly_hfrr

sp5 = SP500Index()
sp5_idx = sp5.return_index
opt_sim = OptionSimulation(update_simulation_data=True)


Option files not updated


# Simulation Parameters
Setup simulation parameters, leverage and simulation index.

In [2]:
duration = 2
leverage = 1
sim_idx = 0
%time sim_output_put = [opt_sim.trade_sim(-0.25, duration, trade_day_type=day_type) \
                    for day_type in ['EOM']]

%time sim_output_call = [opt_sim.trade_sim(0.25, duration, trade_day_type=day_type, option_type='C') \
                    for day_type in ['EOM']]

Running Simulation - trade_day_type:EOM | Z-score -0.25 | Duration 2 | Option Type:P
CPU times: user 38.9 s, sys: 2.38 s, total: 41.3 s
Wall time: 38.5 s
Running Simulation - trade_day_type:EOM | Z-score 0.25 | Duration 2 | Option Type:C
CPU times: user 43.7 s, sys: 8.88 s, total: 52.6 s
Wall time: 53.3 s


In [3]:
opt_trade_put, opt_trade_call = [OptionTrades(sim_output, leverage=leverage) for sim_output in [sim_output_put[0],
                                                                                               sim_output_call[0]]]
put_returns, call_returns = [opt_trade.returns for opt_trade in [opt_trade_put,
                                                                opt_trade_call]]
                                 
put_idx, call_idx = [opt_trade.return_index for opt_trade in [opt_trade_put,
                                                                opt_trade_call]]

put_delta, call_delta = [opt_trade.greeks['delta_1545'] for opt_trade in [opt_trade_put,
                                                                opt_trade_call]]

In [14]:
stradle_idx = put_idx * call_idx
straddle_ret = call_returns + put_returns
straddle_ret = straddle_ret.rename('ATM_Straddle')
perf = [perf_stats(strat) for strat in [straddle_ret, put_returns, call_returns]]

straddle_delta = put_delta + call_delta
straddle_delta_pos = straddle_delta > 0.1
straddle_delta_neg = straddle_delta < -0.1
straddle_delta_tmp = straddle_delta.copy()
straddle_delta_tmp[~(straddle_delta_pos.values + straddle_delta_neg.values)] = 0


sp5_idx = sp5_idx.reindex(stradle_idx.index)
hedge_ret = sp5_idx.pct_change() * straddle_delta_tmp.shift(1)
hedge_ret = hedge_ret.dropna(how='any')
strat_ret = straddle_ret - hedge_ret
strat_perf = perf_stats(strat_ret)
strat_perf = strat_perf.rename('Hedged Straddle')
strat_idx = pf.timeseries.cum_returns(strat_ret, 100)
pd.concat([pd.concat(perf, axis=1), strat_perf], axis=1)


,ATM_Straddle,EOMP-0.25L1.0,EOMC0.25L1.0,Hedged Straddle
StartDate,"Feb 02, 2004","Feb 02, 2004","Feb 02, 2004","Feb 02, 2004"
EndDate,"Apr 02, 2019","Apr 02, 2019","Apr 02, 2019","Apr 02, 2019"
annual_return,0.0770273,0.0715138,0.00265729,0.0293288
annual_volatility,0.0968732,0.10573,0.0561083,0.0527606
sharpe_ratio,0.814758,0.706506,0.0753744,0.574423
calmar_ratio,0.331166,0.233211,0.00903753,0.163932
stability_of_timeseries,0.988125,0.979878,0.240358,0.96511
max_drawdown,-0.232594,-0.306648,-0.294028,-0.178909
omega_ratio,1.2337,1.1902,1.01479,1.15319
sortino_ratio,1.13755,0.968325,0.104223,0.75498
